In [1]:
%%file model.py

from accelerate import Accelerator
accelerator = Accelerator()   # no mixed_precision

import os
import bisect
from PIL import Image, UnidentifiedImageError

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torchvision.transforms import v2 as T
from tqdm import tqdm

# -----------------------------
# Dataset
# -----------------------------



def save_last_checkpoint(epoch, model, optimizer, best_val_loss, path="last_checkpoint.pth"):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "best_val_loss": best_val_loss,
    }, path)


def load_checkpoint(path, model, optimizer, device):
    if os.path.exists(path):
        print("🔄 Found checkpoint. Resuming training...")
        checkpoint = torch.load(path, map_location=device)

        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

        start_epoch = checkpoint["epoch"] + 1
        best_val_loss = checkpoint["best_val_loss"]

        print(f"✅ Resumed from epoch {start_epoch}")
        return start_epoch, best_val_loss
    else:
        print("🚀 No checkpoint found. Starting fresh.")
        return 0, float("inf")






class DrowsySequenceDataset(Dataset):
    def __init__(self, root_dir, seq_len=16, transform=None):
        self.seq_len = seq_len
        self.transform = transform

        self.video_info = []
        self.cumulative_windows = []

        total_windows = 0
        print("[Dataset] Indexing dataset...")

        for subject in os.listdir(root_dir):
            subject_path = os.path.join(root_dir, subject)
            if not os.path.isdir(subject_path):
                continue

            for scenario in os.listdir(subject_path):
                scenario_path = os.path.join(subject_path, scenario)
                if not os.path.isdir(scenario_path):
                    continue

                frame_files = sorted([
                    f for f in os.listdir(scenario_path)
                    if f.lower().endswith((".jpg", ".jpeg", ".png"))
                ])

                num_frames = len(frame_files)
                if num_frames < self.seq_len:
                    continue

                num_windows = num_frames - self.seq_len + 1
                total_windows += num_windows
                self.video_info.append((scenario_path, frame_files, num_windows))
                self.cumulative_windows.append(total_windows)

        self.total_windows = total_windows
        print(f"[Dataset] Total scenarios: {len(self.video_info)}")
        print(f"[Dataset] Total sequences: {self.total_windows}")

    def __len__(self):
        return self.total_windows

    def __getitem__(self, idx):
        video_idx = bisect.bisect_right(self.cumulative_windows, idx)

        window_start = idx if video_idx == 0 else idx - self.cumulative_windows[video_idx - 1]
        video_path, frame_files, _ = self.video_info[video_idx]

        images = []
        frame_slice = frame_files[window_start:window_start + self.seq_len]
        frame_labels = []

        for fname in frame_slice:
            img_path = os.path.join(video_path, fname)
            try:
                img = Image.open(img_path).convert("L")  # Grayscale
                if self.transform:
                    img = self.transform(img)
                if isinstance(img, torch.Tensor) and img.ndim == 2:
                    img = img.unsqueeze(0)
                images.append(img)

                # Label extraction: last underscore before extension
                label = int(fname.rsplit("_", 1)[-1].split(".")[0])
                frame_labels.append(label)

            except (UnidentifiedImageError, OSError):
                images.append(torch.zeros(1, 224, 224))
                frame_labels.append(0)

        images = torch.stack(images)  # (T, C, H, W)
        # Aggregate labels: Drowsy if any frame is drowsy
        label = int(any(frame_labels))

        return images, torch.tensor(label, dtype=torch.long)

# -----------------------------
# CNN Encoder (MobileNetV3 Small)
# -----------------------------
class CNNEncoder(nn.Module):
    def __init__(self, feature_dim=512, pretrained=True):
        super().__init__()
        weights = MobileNet_V3_Small_Weights.DEFAULT
        mobilenet = mobilenet_v3_small(weights=weights)

        # Replace first conv for 1-channel input
        old_conv = mobilenet.features[0][0]
        mobilenet.features[0][0] = nn.Conv2d(
            in_channels=1,
            out_channels=old_conv.out_channels,
            kernel_size=old_conv.kernel_size,
            stride=old_conv.stride,
            padding=old_conv.padding,
            bias=False
        )
        with torch.no_grad():
            mobilenet.features[0][0].weight.copy_(old_conv.weight.mean(dim=1, keepdim=True))

        self.features = mobilenet.features
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(576, feature_dim)

        # Freeze all MobileNet weights (can unfreeze last layers if needed)
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.fc(x)
        return x  # (B*T, feature_dim)

# -----------------------------
# Causal TCN Block
# -----------------------------
class CausalTemporalConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation

        self.conv = nn.Conv1d(
            in_channels, out_channels, kernel_size,
            padding=self.padding, dilation=dilation
        )
        self.norm = nn.GroupNorm(num_groups=8, num_channels=out_channels)
        self.relu = nn.GELU()
        self.residual = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        out = self.conv(x)
        if self.padding > 0:
            out = out[:, :, :-self.padding]  # causal trimming
        out = self.norm(out)
        out = self.relu(out)
        return out + self.residual(x)

# -----------------------------
# Full CNN + TCN Model
# -----------------------------
class CNNTemporalConv(nn.Module):
    def __init__(self, feature_dim=512):
        super().__init__()
        self.encoder = CNNEncoder(feature_dim)
        self.tcn = nn.Sequential(
            CausalTemporalConvBlock(feature_dim, 256, kernel_size=3, dilation=1),
            CausalTemporalConvBlock(256, 256, kernel_size=3, dilation=2),
            CausalTemporalConvBlock(256, 128, kernel_size=3, dilation=4),
        )
        self.classifier = nn.Linear(128, 1)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B*T, C, H, W)
        features = self.encoder(x)           # (B*T, F)
        features = features.view(B, T, -1)   # (B, T, F)
        features = features.transpose(1, 2)  # (B, F, T)
        tcn_out = self.tcn(features)         # (B, 128, T)
        last_feature = tcn_out[:, :, -1]     # (B, 128)
        logits = self.classifier(last_feature)
        return logits.squeeze(1)             # (B,)

# -----------------------------
# Transforms
# -----------------------------
transform = T.Compose([
    T.ToImage(),                        # PIL → Tensor
    T.ToDtype(torch.float32, scale=1/255),
    T.Normalize(mean=[0.5], std=[0.5])  # Grayscale normalization
])

# -----------------------------
# Datasets & Loaders
# -----------------------------


# train_ds1 = DrowsySequenceDataset("/root/.cache/kagglehub/datasets/manith04/ddd-processed-1-training-frames-type-1/versions/1", transform=transform)
# train_ds2 = DrowsySequenceDataset("/root/.cache/kagglehub/datasets/manith04/ddd-processed-2-training-frames-type-1/versions/1", transform=transform)
# val_ds = DrowsySequenceDataset("/root/.cache/kagglehub/datasets/manith04/ddd-processed-validation-frames-type-1/versions/1", transform=transform)

train_ds1 = DrowsySequenceDataset("/kaggle/input/datasets/manith04/ddd-processed-1-training-frames-type-1", transform=transform)
train_ds2 = DrowsySequenceDataset("/kaggle/input/datasets/manith04/ddd-processed-2-training-frames-type-1", transform=transform)
val_ds = DrowsySequenceDataset("/kaggle/input/datasets/manith04/ddd-processed-validation-frames-type-1", transform=transform)


train_dataset = ConcatDataset([train_ds1, train_ds2])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# -----------------------------
# Device, Model, Optimizer
# -----------------------------
model = CNNTemporalConv()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


epochs = 51
checkpoint_path = "last_checkpoint.pth"
best_model_path = "best_model.pth"



model, optimizer, train_loader, val_loader = accelerator.prepare(model, optimizer, train_loader, val_loader)

if os.path.exists(checkpoint_path):
    start_epoch, best_val_loss = load_checkpoint(checkpoint_path, model, optimizer, accelerator.device)
else:
    start_epoch = 0
    best_val_loss = float("inf")



for epoch in range(start_epoch, epochs):
    print("\n" + "="*60)
    print(f"Epoch {epoch+1}/{epochs}")
    print("="*60)

    model.train()
    train_loss = 0
    progress_bar = tqdm(train_loader, desc="Training", leave=False)

    for frames, labels in progress_bar:
        labels = labels.float()
        outputs = model(frames)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        accelerator.backward(loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        train_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})

    avg_train_loss = train_loss / len(train_loader)

    # --------- VALIDATION ----------
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for frames, labels in tqdm(val_loader, desc="Validation", leave=False):
            labels = labels.float()
            outputs = model(frames)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = (torch.sigmoid(outputs) > 0.5).int()
            correct += (preds == labels.int()).sum().item()
            total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total

    # --------- SUMMARY ----------
    print(f"Train Loss : {avg_train_loss:.4f}")
    print(f"Val Loss   : {avg_val_loss:.4f}")
    print(f"Val Acc    : {val_acc:.4f}")

    # with open("experiment.txt", "a") as file:
    #     file.write(avg_train_loss, )
        

    # --------- SAVE BEST MODEL ----------
    if accelerator.is_main_process:
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            unwrapped_model = accelerator.unwrap_model(model)
            torch.save(unwrapped_model.state_dict(), best_model_path)
            print("⭐ Best model updated!")

        # Always save last checkpoint
        save_last_checkpoint(epoch, model, optimizer, best_val_loss, checkpoint_path)

print("\n🎉 Training Complete.")


Overwriting model.py


In [ ]:
!accelerate launch model.py --no-processes=2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956[Dataset] Indexing dataset...

[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 20
[Dataset] Total sequences: 172

In [ ]:
!accelerate launch model.py --no-processes=2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Dataset] Indexing dataset...[Dataset] Indexing dataset...

[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total scenarios: 40[Dataset] Total sequences: 326956

[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 20
[Dataset] Total sequences: 172

In [ ]:
!accelerate launch model.py --no-processes=2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956[Dataset] Total scenarios: 49

[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...[Dataset] Indexing dataset...

[Dataset] Total scenarios: 40[Dataset] Total scenarios: 40

[Dataset] Total sequences: 326956[Dataset] Total sequences: 326956

[Dataset] Indexing dataset...[Dataset] Indexing dataset...

[Dataset] Total scenarios: 20
[Dataset] Total sequences: 172

In [ ]:
!accelerate launch model.py --no-processes=2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 20
[Dataset] Total sequences: 172

In [ ]:
!accelerate launch model.py --no-processes=2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 20
[Dataset] Total sequences: 172

In [ ]:
!accelerate launch model.py --no-processes=2

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 49
[Dataset] Total sequences: 394956
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 40[Dataset] Total scenarios: 40
[Dataset] Total sequences: 326956

[Dataset] Total sequences: 326956
[Dataset] Indexing dataset...
[Dataset] Indexing dataset...
[Dataset] Total scenarios: 20
[Dataset] Total sequences: 172